# Mesh Analysis

PolyMesh provides some useful classes and algorithms for mesh management and operations related to the geometry or the topology of polygonal meshes.

In [1]:
import pyvista as pv
import numpy as np
from polymesh import PolyData

d, h, a = 6.0, 15.0, 15.0
cyl = pv.CylinderStructured(
    center=(0.0, 0.0, 0.0),
    direction=(0.0, 0.0, 1.0),
    radius=np.linspace(d / 2, a / 2, 15),
    height=h,
    theta_resolution=100,
    z_resolution=40,
)
mesh = PolyData.from_pv(cyl)

## Nodal Distribution Factors

In [2]:
mesh.nodal_distribution_factors()

array([[1.        , 0.47580645, 0.23790323, ..., 0.23790323, 0.11895161,
        0.25      ],
       [0.52419355, 0.47794118, 0.23897059, ..., 0.23897059, 0.11948529,
        0.13104839],
       [0.52205882, 0.47972973, 0.23986486, ..., 0.23986486, 0.11993243,
        0.13051471],
       ...,
       [0.12807377, 0.12207031, 0.24414062, ..., 0.24414062, 0.48828125,
        0.51229508],
       [0.12792969, 0.12220149, 0.24440299, ..., 0.24440299, 0.48880597,
        0.51171875],
       [0.12779851, 0.25      , 0.5       , ..., 0.5       , 1.        ,
        0.51119403]])

## Nodal Adjacency

To access the nodal adjacency matrix as a scipy sparse matrix:

In [3]:
mesh.nodal_adjacency_matrix(frmt="scipy-csr")

<60600x60600 sparse array of type '<class 'numpy.intc'>'
	with 1527274 stored elements in Compressed Sparse Row format>

as a NetworkX graph:

In [4]:
mesh.nodal_adjacency_matrix(frmt="nx")

as an `Awkward` array:

In [5]:
mesh.nodal_adjacency_matrix(frmt="jagged")

<Array [[0, 1, 15, 16, ..., 1516, 1530, 1531], ...] type='60600 * var * int64'>

or as a Numba-jittable CSR matrix from `Neumann`:

In [6]:
mesh.nodal_adjacency_matrix(frmt="csr")

60600x60600 CSR matrix of 1527274 values.

## Pseudo Peripheral Nodes

In [7]:
from neumann.topology.graph import pseudo_peripheral_nodes

csr = mesh.nodal_adjacency_matrix(frmt="csr")
ppn = pseudo_peripheral_nodes(csr)

## Rooted Level Structure

In [8]:
from neumann.topology.graph import rooted_level_structure

rls = rooted_level_structure(csr)

## K-Nearest-Neighbours with ``Scipy`` or ``SkLearn``

In [9]:
from polymesh.grid import Grid

size = 80, 60, 20
shape = 10, 8, 4
grid = Grid(size=size, shape=shape, eshape="H8")
grid.k_nearest_cell_neighbours(k=3, knn_options=dict(max_distance=10.0))[:5]

array([[0, 1, 4],
       [1, 2, 0],
       [2, 1, 3],
       [3, 2, 7],
       [4, 5, 0]], dtype=int64)